In [ ]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# プログラミングのない世界 (2)

## 自動的にプログラムを作成する (論理合成、定理証明)

* [Logic synthesis - Wikipedia](https://en.wikipedia.org/wiki/Logic_synthesis)
* [Automated theorem proving - Wikipedia](https://en.wikipedia.org/wiki/Automated_theorem_proving)

剰余 ``divmod`` と等価な論理演算

In [16]:
import itertools

In [68]:
for i,j in itertools.product([0,1], repeat=2):
    print("{:d}+{:d}={:02b}".format(i,j,i+j))

0+0=00
0+1=01
1+0=01
1+1=10


In [106]:
for i,j in itertools.product([0,1], repeat=2):
    print("{:d}+{:d}={:02b} ".format(i,j,i+j), end="")
    print("({:d},{:d})".format(*divmod(i+j,2)))

0+0=00 (0,0)
0+1=01 (0,1)
1+0=01 (0,1)
1+1=10 (1,0)


### Boolean Function
* [Logic — SymPy 1.10.1 documentation](https://docs.sympy.org/latest/modules/logic.html#boolean-functions)
  * [BooleanFunction—Wolfram Language Documentation](https://reference.wolfram.com/language/ref/BooleanFunction.html.en)

前回、繰り上がりについて ``not(a^b)&(a|b)`` などとコーディングしてしまったが、簡約化すると ``a&b``

In [ ]:
def binary_add(a, b):
    if not (a >=0 and b >= 0 and a < 2 and b < 2):
        raise ValueError
    # YOUR CODE HERE
    #return [(a+b)//2, (a+b)%2]
    #return list(divmod(a+b, 2))
    return [(not(a^b))&(a|b), a^b]

In [2]:
from sympy.abc import x,y,z
from sympy.logic import simplify_logic

In [42]:
simplify_logic(~(x^y)&(x|y))

x & y

In [110]:
for i,j in itertools.product([0,1], repeat=2):
    print("{:d}+{:d}={:02b} ".format(i,j,i+j), end="")
    print("({:d},{:d})".format(i&j, i^j))

0+0=00 (0,0)
0+1=01 (0,1)
1+0=01 (0,1)
1+1=10 (1,0)


すなわち、`divmod(i+j,2)` と `(i&j, i^j)` は「相等 (equal)」
* [Function (mathematics) - Wikipedia](https://en.wikipedia.org/wiki/Function_%28mathematics%29)

### CNF and DNF, ANF

論理演算には相等な表現が無数に存在する。ある規約の下で「標準形」へ簡略化が可能:

* [Conjunctive normal form - Wikipedia](https://en.wikipedia.org/wiki/Conjunctive_normal_form)
  - "**P**roduct **o**f **S**ums"
  - "**an AND of ORs**"
  - [Logical conjunction - Wikipedia](https://en.wikipedia.org/wiki/Logical_conjunction)
    - 「連言 (論理積)」
* [Disjunctive normal form - Wikipedia](https://en.wikipedia.org/wiki/Disjunctive_normal_form)
  - "**S**um **o**f **P**roducts""
  - "**an OR of ANDs**"
  - [Logical disjunction - Wikipedia](https://en.wikipedia.org/wiki/Logical_disjunction)
    - 「選言 (論理和)」
* [Algebraic normal form - Wikipedia](https://en.wikipedia.org/wiki/Algebraic_normal_form)
  - One or more variables are **AND**ed together into a *term*,
  - then one or more terms are **XOR**ed together into *ANF*.
  - *No **NOT**s are permitted*

論理演算関数 (Boolean Function) では、出力が決まれば論理演算を決定できる

In [122]:
list(itertools.product([0,1],repeat=2))

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [30]:
from sympy.logic.boolalg import ANFform, to_cnf, to_dnf

In [118]:
ANFform([x,y], [0,0,0,0])

False

In [119]:
ANFform([x,y], [1,1,1,1])

True

In [123]:
ANFform([x,y], [0,0,0,1])

x & y

In [125]:
ANFform([x,y], [0,1,1,0])

x ^ y

In [124]:
ANFform([x,y], [0,1,1,1])

x ^ y ^ (x & y)

In [6]:
simplify_logic(x^y^(x&y), form='cnf')

x | y

In [7]:
simplify_logic(x^y^(x&y), form='dnf')

x | y

まとめ)

In [114]:
import operator

In [115]:
for op in [operator.and_, operator.or_, operator.xor]:
    print([op(i,j) for i,j in itertools.product([0,1],repeat=2)])

[0, 0, 0, 1]
[0, 1, 1, 1]
[0, 1, 1, 0]


In [117]:
for op in [operator.and_, operator.or_, operator.xor]:
    print(ANFform([x,y], [op(i,j) for i,j in itertools.product([0,1],repeat=2)]))

x & y
x ^ y ^ (x & y)
x ^ y


## 一桁の二進数の加算と等価な論理演算

In [106]:
for i,j in itertools.product([0,1], repeat=2):
    print("{:d}+{:d}={:02b} ".format(i,j,i+j), end="")
    print("({:d},{:d})".format(*divmod(i+j,2)))

0+0=00 (0,0)
0+1=01 (0,1)
1+0=01 (0,1)
1+1=10 (1,0)


In [103]:
list(zip(*[divmod(i+j,2) for i,j in itertools.product([0,1], repeat=2)]))

[(0, 0, 0, 1), (0, 1, 1, 0)]

In [105]:
for qr in zip(*[divmod(i+j,2) for i,j in itertools.product([0,1], repeat=2)]):
    print(ANFform([x,y], qr))

x & y
x ^ y


xor (``^``) を用いずに CNF や DNF で表現すると

In [40]:
for qr in zip(*[divmod(i+j,2) for i,j in itertools.product([0,1], repeat=2)]):
    print(to_cnf(ANFform([x,y], qr), simplify=True))

x & y
(x | y) & (~x | ~y)


In [39]:
for qr in zip(*[divmod(i+j,2) for i,j in itertools.product([0,1], repeat=2)]):
    print(to_dnf(ANFform([x,y], qr), simplify=True))

x & y
(x & ~y) | (y & ~x)


### 繰り上がりを考慮した一桁の二進数の加算

In [17]:
import itertools
for i,j,k in itertools.product([0,1], repeat=3):
    print("{:d}+{:d}+{:d}={:02b} ".format(i,j,k,i+j+k), end="")
    print("({:d},{:d})".format(*divmod(i+j+k,2)))

0+0+0=00 (0,0)
0+0+1=01 (0,1)
0+1+0=01 (0,1)
0+1+1=10 (1,0)
1+0+0=01 (0,1)
1+0+1=10 (1,0)
1+1+0=10 (1,0)
1+1+1=11 (1,1)


In [20]:
list(zip(*[divmod(i+j+k,2) for i,j,k in itertools.product([0,1], repeat=3)]))

[(0, 0, 0, 1, 0, 1, 1, 1), (0, 1, 1, 0, 1, 0, 0, 1)]

In [22]:
for qr in zip(*[divmod(i+j+k,2) for i,j,k in itertools.product([0,1], repeat=3)]):
    print(ANFform([x,y,z], qr))

(x & y) ^ (x & z) ^ (y & z)
x ^ y ^ z


xor (``^``) を用いずにCNFやDNFで表現すると煩雑な表現になる (xor演算がが重用される理由) :

In [36]:
for qr in zip(*[divmod(i+j+k,2) for i,j,k in itertools.product([0,1], repeat=3)]):
    print(to_cnf(ANFform([x,y,z], qr), simplify=True))

(x | y) & (x | z) & (y | z)
(x | y | z) & (x | ~y | ~z) & (y | ~x | ~z) & (z | ~x | ~y)


In [35]:
for qr in zip(*[divmod(i+j+k,2) for i,j,k in itertools.product([0,1], repeat=3)]):
    print(to_dnf(ANFform([x,y,z], qr), simplify=True))

(x & y) | (x & z) | (y & z)
(x & y & z) | (x & ~y & ~z) | (y & ~x & ~z) | (z & ~x & ~y)
